In [1]:
import os
from bs4 import BeautifulSoup
from itertools import chain
import pandas as pd
from openai import OpenAI
import re 
import tiktoken  # для подсчета токенов
import numpy as np
from collections import defaultdict

from make_embeding_from_docs import *


In [97]:
from bs4 import BeautifulSoup
from collections import defaultdict

def is_tag(element):
    """Проверяет, является ли элемент тегом."""
    return element.name is not None

def is_header(element):
    """Проверяет, является ли элемент заголовком."""
    return element.name.startswith('h') and element.name[1:].isdigit()

def contains_heading_tags(element):
    """Проверяет, содержит ли элемент вложенные заголовки."""
    return bool(element.find(is_header))

def update_header_hierarchy(hierarchy, header):
    """Обновляет иерархию заголовков."""
    header_level = int(header.name[-1])  # Уровень текущего заголовка (1 для h1, 2 для h2 и т.д.)
    header_text = header.get_text(strip=True)  # Текст текущего заголовка

    # Удаляем все заголовки с уровнем >= текущего
    for level in list(hierarchy.keys()):  # Используем list для безопасного удаления
        if level >= header_level:
            del hierarchy[level]

    # Добавляем текущий заголовок (уровень и текст)
    hierarchy[header_level] = header_text

def process_element(element, hierarchy=None, sections=None) -> dict[int:str]:
    if hierarchy is None:
        hierarchy = {}  # Инициализация иерархии как словаря
    if sections is None:
        sections = defaultdict(str)  # Инициализация словаря с пустыми строками по умолчанию

    while element is not None:
        if is_tag(element):
            if is_header(element):
                update_header_hierarchy(hierarchy, element)
            else:
                if contains_heading_tags(element):
                    first_child = element.find()
                    process_element(first_child, hierarchy, sections)
                else:
                    # Создаем ключ для sections, используя значения hierarchy
                    hierarchy_key = tuple(hierarchy.values())
                    text = element.get_text(strip=True,separator='\n')
                    sections[hierarchy_key] += "\n" + text if sections[hierarchy_key] else text
        else:
            text = element.strip()
            if text:
                # Создаем ключ для sections, используя значения hierarchy
                hierarchy_key = tuple(hierarchy.values())
                sections[hierarchy_key] += "\n" + text if sections[hierarchy_key] else text
        element = element.next_sibling

    return sections

# Новый HTML-документ
html_doc = """
<html>
<head><title>Test Page</title></head>
<body>
    <div id="main">
        <h1>Заголовок 1</h1>
        <p>Paragraph 1</p>
        <div>
            <h2>Заголовок 2</h2>
            <p>Paragraph inside div</p>
        </div>
        <span>Span 1</span>
        <h2>Другой заголовок 2</h2>
        Текстовый узел после span
        <p>Paragraph 3</p>
        <a href="#">Link</a>
        <h1>Другой заголовок 1</h1>
        <h3>Другой заголовок 3</h3>
        <p>Paragraph 1</p>
    </div>
</body>
</html>
"""

soup = BeautifulSoup(html_doc, 'html.parser')

# Пример 1: Начинаем с элемента <div id="main">
element = soup.find('div', id='main')
sections = process_element(element)
sections

defaultdict(str,
            {('Заголовок 1',): 'Paragraph 1',
             ('Заголовок 1', 'Заголовок 2'): 'Paragraph inside div\nSpan 1',
             ('Заголовок 1',
              'Другой заголовок 2'): 'Текстовый узел после span\nParagraph 3\nLink',
             ('Другой заголовок 1', 'Другой заголовок 3'): 'Paragraph 1'})

In [98]:
def all_section_with_text(html,preheader):
    soup = BeautifulSoup(html, "html.parser")
    hierarchy = {0:preheader}
    element = soup.find()
    sections_with_text = [[list(sections), text] for sections, text in process_element(element,hierarchy=hierarchy).items()]
    return sections_with_text

html = html_doc
preheader = 'Seller'
all_section_with_text(html,preheader)

[[['Seller'], 'Test Page'],
 [['Seller', 'Заголовок 1'], 'Paragraph 1'],
 [['Seller', 'Заголовок 1', 'Заголовок 2'], 'Paragraph inside div\nSpan 1'],
 [['Seller', 'Заголовок 1', 'Другой заголовок 2'],
  'Текстовый узел после span\nParagraph 3\nLink'],
 [['Seller', 'Другой заголовок 1', 'Другой заголовок 3'], 'Paragraph 1']]

In [100]:

MAX_TOKENS = 1600
SAVE_PATH = "./embeddings.csv"

doc_folder = 'ozon docs'
doc_names_and_preheaders = [
    ['seller.html','Ozon Seller API']
    ,['performance.html','Ozon Performance API']
]

preheaders_and_html = []
for name, preheader in doc_names_and_preheaders:
    path = os.path.join(doc_folder,name)
    with open(path, "r", encoding="utf-8") as file:
        html = file.read()
    preheaders_and_html.append([preheader,html])

sections = []
for preheader, html in preheaders_and_html:
    sections += all_section_with_text(html,preheader)
sections

sections = [clean_section(ws) for ws in sections]
sections = [ws for ws in sections if keep_section(ws)]

strings = []
for section in sections:
    strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS, max_recursion=7))
sections = strings

df = pd.DataFrame({"text": sections[:10]})
# # df = pd.DataFrame({"text": sections})

# # df['embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

df.to_csv(SAVE_PATH, index=False)